* Redis的基本介绍，只展示代码，不展示介绍
* 需要启动redis

In [1]:
import redis
conn = redis.Redis()

# Redis 中的字符串

In [2]:
conn.set('hello','world')

True

In [3]:
conn.get('hello')

b'world'

In [4]:
conn.get('hello').decode()

'world'

In [5]:
conn.delete('hello')

1

In [6]:
conn.get('hello')

In [7]:
conn.get('hello') is None

True

# Reids 中的列表

In [8]:
conn.rpush('list-key', 'item')

1

In [9]:
conn.rpush('list-key', 'item2', 'item')

3

In [10]:
conn.lrange('list-key', 0, -1)

[b'item', b'item2', b'item']

In [11]:
conn.lpop('list-key')

b'item'

后续还有很多例如在指定位置插入数据之类的操作，以后再了解

# Redis 中的集合

In [12]:
conn.sadd('set-key', 'item')

1

In [13]:
# 值表示的是新增的数量
conn.sadd('set-key', 'item2', 'item3', 'item')

2

In [14]:
conn.sismember('set-key', 'item')

True

In [15]:
conn.smembers('set-key')

{b'item', b'item2', b'item3'}

In [16]:
conn.srem('set-key', 'item')

1

In [17]:
conn.srem('set-key', 'item')

0

In [18]:
conn.smembers('set-key')

{b'item2', b'item3'}

当然还有交集或者补集之类的操作

# Redis 散列

In [19]:
conn.hset('hash-key', 'sub-key1', 'value1')

1

In [20]:
# 表示新增的量
conn.hset('hash-key', 'sub-key2', 'value2')

1

In [21]:
# 散列的键是唯一的
conn.hset('hash-key', 'sub-key1', 'value1')

0

In [22]:
conn.hgetall('hash-key')

{b'sub-key1': b'value1', b'sub-key2': b'value2'}

In [23]:
conn.hdel('hash-key', 'sub-key2')

1

In [24]:
conn.hdel('hash-key', 'sub-key2')

0

In [25]:
conn.hget('hash-key', 'sub-key2')

In [26]:
conn.hget('hash-key', 'sub-key1')

b'value1'

In [27]:
conn.hgetall('hash-key')

{b'sub-key1': b'value1'}

# Redis有序集合

有序集合和散列一样，都用于存储键值对
* 有序集合的键被称为成员(member)，每个成员都是各不相同的
* 有序集合的值被称为分值(score)，必须为浮点数

唯一一个既可以根据成员访问元素，又可以根据分值以及分值的排列顺序来访问元素的结构

In [28]:
conn.zadd('zset-key', 'member1', 728)

1

In [29]:
conn.zadd('zset-key', 'member0', 982)

1

In [30]:
conn.zadd('zset-key', 'member0', 982)

0

In [31]:
conn.zrange('zset-key', 0, -1)

[b'member1', b'member0']

In [32]:
conn.zrange('zset-key', 0, -1, withscores=True)

[(b'member1', 728.0), (b'member0', 982.0)]

In [33]:
conn.zrangebyscore('zset-key', 0, 800, withscores=True)

[(b'member1', 728.0)]

In [34]:
conn.zrem('zset-key', 'member1')

1

In [35]:
conn.zrem('zset-key', 'member1')

0

In [36]:
conn.zrange('zset-key', 0, -1)

[b'member0']

# 解决实际问题

## 对文章进行投票
* 通过散列实现对文章信息的存储
* 通过有序集合实现对文章id按照发布时间与评分顺序的存储
* 通过集合实现每人一票的控制
* 对投票开放时间做一个限制

In [37]:
import time
# 常量
ONE_WEEK_IN_SECONDS = 7 * 86400
VOTE_SCORE = 432

In [38]:
# 文章评分函数
def article_vote(conn, user, article):
    cutoff = time.time() - ONE_WEEK_IN_SECONDS
    if conn.zscore('time:', article) < cutoff:
        return
    article_id = article.partition(':')[-1]
    # 如果用户是第一次为这篇文章增加评分
    if conn.sadd('voted:' + article_id, user):
        conn.zincrby('score:', article, VOTE_SCORE)
        conn.hincrby(article, 'votes', 1)

## 发布并获取文章
* 通过INCR获得文章id， 加入到集合
* 通过EXPIRE为集合设置一个过期时间
* HMSET为文章加入信息
* ZADD赋予初始分和发布时间

In [39]:
def post_article(conn, user, title, link):
    article_id = str(conn.incr('article:'))
    voted = 'voted:' + article_id
    # 将自己加入投票集合
    conn.sadd(voted, user)
    # 设置一周的过期时间
    conn.expire(voted, ONE_WEEK_IN_SECONDS)
    now = time.time()
    article = 'article:' + article_id
    conn.hmset(article,{
        'title': title,
        'link': link,
        'poster': user,
        'time': now,
        'votes':1
    })
    conn.zadd('score:', article, now + VOTE_SCORE)
    conn.zadd('time:', article, now)
    return article_id

In [40]:
# 获取文章
ARTICLE_PER_PAGE = 25
def get_articles(conn, page, order='score:'):
    start = (page - 1) * ARTICLE_PER_PAGE
    end = start + ARTICLE_PER_PAGE
    # 获取多个文章id
    ids = conn.zrevrange(order, start, end)
    articles = list()
    for id in ids:
        article_data = conn.hgetall(id)
        # article_data 中不包含 article_id
        artical_data['id'] = id
        articles.append(article_data)
    return articles

## 对文章进行分组

In [41]:
# 将文章添加或移除到某个群组
def add_remove_groups(conn, article_id, to_add=[], to_remove=[]):
    article = 'article:' + article_id
    for group in to_add:
        conn.sadd('group:' + group, article)
    for group in to_remove:
        conn.srem('group:' + group, article)

为了实现对群组文章进行排序和粉雪的操作，网站需要将同一个群组里面的所有文章都按照评分有序的存储到一个有序集合里。  
此时可以用到 ZINTERSTORE 功能，会取到集合或者有序集合中的交集，并且按照自定义的方法形成一个有序集合  
所谓的自定义，也就是定义分值。 每个集合的分值看做是1， 因此我们在这里选择最大值进行排序。

In [42]:
def get_group_articles(conn, group, page, order='score:'):
    # 命名
    key = order + group
    if not conn.exists(key):
        conn.zinterstore(key,
                        ['group:' + group, order],
                        aggregate = 'max')
        conn.expire(key, 60)
    return get_articles(conn, page, key)